In [11]:
# Emerging Technology Tasks

# Task 1: Building a trigram model from Project Gutenberg books

# This project reads and processes five books from Project Gutenberg to build a trigram model.
# A trigram is a sequence of three characters
# The trigram model counts how often each trigram appears in the text.

import re
from collections import defaultdict

In [12]:
# Paths to the five text files
# These represent the text files of the books to be processed
file_paths = ['Book1.txt', 'Book2.txt', 'Book3.txt',  'Book4.txt',  'Book5.txt'   
]

In [13]:
# Function to read text from a file
# This function opens the file, reads the entire content, and returns it as a string
def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [14]:
# Function for cleaning text up
# This function removes unnecessary parts of the text, like the preamble and postamble
# It also removes all characters such as ASCII letters, spaces, full stops, 
# It also converts the text to uppercase
def clean_text(text):
    # Remove preamble and postamble
    start = re.search(r'\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    end = re.search(r'\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    if start and end:
        text = text[start.end():end.start()]

    # Remove all non-ASCII letters except full stops and spaces
    text = re.sub(r'[^A-Za-z. ]', '', text)

    # Convert all letters to uppercase for consistency
    text = text.upper()

    return text

In [15]:
# Function to create a trigram model
# This function takes the cleaned text and creates a trigram model
# It counts how many times each sequence of three characters appears in the text
def create_trigram_model(text):
    # Use a defaultdict to store trigram counts. Default value is 0 for any trigram not yet encountered
    trigram_model = defaultdict(int)
    
    # Slide through the text to create trigrams and count their occurrences
    for i in range(len(text) - 2):
        trigram = text[i:i+3]  # Extract a sequence of 3 characters 
        trigram_model[trigram] += 1  # Increment the count for this trigram

    return trigram_model

In [16]:
# Read and clean texts from all files
# This reads and cleans each of the five books, removing unwanted characters and standardizing the format.
texts = [clean_text(read_text(file_path)) for file_path in file_paths]

# Combine all cleaned texts into one
# Joins the texts from all the books into a single large block of text.
combined_text = ' '.join(texts)

# Create the trigram model using the combined text
# This generates the trigram model, counting how often each trigram appears.
trigram_model = create_trigram_model(combined_text)

In [ ]:
# Output some of the trigram model
# Here, we print the first 10 trigrams and their counts from the model to see the results.
for trigram, count in list(trigram_model.items())[:10]:
    print(f'{trigram}: {count}')

In [ ]:
# Task 2
import random

def get_next_char(trigram_model, last_two_chars):
    # Find all trigrams that start with the last two characters
    matching_trigrams = {trigram: count for trigram, count in trigram_model.items() if trigram.startswith(last_two_chars)}
    
    if not matching_trigrams:
        return None  # If no matching trigrams, return None
    
    # Get the third characters and their counts
    third_chars = [trigram[2] for trigram in matching_trigrams.keys()]
    counts = list(matching_trigrams.values())
    
    # Select the next character using the counts as weights
    next_char = random.choices(third_chars, weights=counts, k=1)[0]
    
    return next_char


def generate_text(trigram_model, length=10000):
    # Start with the string "TH"
    generated_text = "TH"
    
    # Generate enough letters till 10000 reached
    while len(generated_text) < length:
        # Get the last two characters of the current string
        last_two_chars = generated_text[-2:]
        
        # Get the next character based on the trigram model
        next_char = get_next_char(trigram_model, last_two_chars)
        
        if next_char is None:
            break  # No matching trigrams found stop the process
        
        # Add the next part to the string
        generated_text += next_char
    
    return generated_text


# Assuming you already have your trigram model from Task 1
generated_text = generate_text(trigram_model, length=10000)

# Print the generated text (you can also save it to a file if needed)
print(generated_text)
